## 0. Import Libraries/Modules

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid

ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (/opt/anaconda3/envs/pic16b/lib/python3.9/site-packages/PIL/__init__.py)